## Подготовка среды

### Установка необходимых библиотек

In [ ]:
#!g1.1
%pip install pyarrow
%pip install transformers
%pip install datasets
%pip install accelerate

In [88]:
#!g1.1
import torch
import torchvision
from torch.utils.data import DataLoader, random_split
import numpy as np
import pandas as pd
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, TextDataset, DataCollatorForLanguageModeling
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import AutoConfig
from transformers import utils
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from datasets import load_dataset, Dataset
import logging
import os

### Загрузка модели и токенайзера



In [ ]:
#!g1.1
# Установка специальных токенов, указываюших на начало и конец последовательности
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2", bos_token="<s>", eos_token="<\s>", pad_token="<pad>", sep_token="<sep>")
model = AutoModelForCausalLM.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2")
model.resize_token_embeddings(len(tokenizer))

# Работа с датасетом

### Загрузка датасета

In [78]:
#!g1.1
# Загружаем датасет из csv файла с помощью Pandas
df = pd.read_csv("Films.csv", encoding='utf-16')
# Добавляем разделяющий токен <sep>
df['Text'] = "Название: " + df['Title'].map(str) + "<sep>" + "Описание: " + df['Description'].map(str)
for i in range(0, 5):
    df['Text'] = df['Text'].apply(lambda x: str(x).replace("  ", " ")[:-1])
df['Text'] = df['Text'].apply(lambda x: str(x).replace("\t", ""))

### Преобразование датасета

In [ ]:
#!g1.1
# Загрузка датасета в объект класс Dataset из пандасовского датафрейма
train_dataset = Dataset.from_pandas(df)
def add_special_token(seq):
    seq["Text"] = '<s>' + seq["Text"] + '<\s>'
    return seq
train_dataset = train_dataset.map(add_special_token)
train_dataset = train_dataset.remove_columns(['Title', 'Description'])
# Кодирование нашего датасета в числовой формат, с помощью токенайзера
def encode(examples):
    return tokenizer(examples['Text'], padding='max_length', max_length=256, truncation=True, return_special_tokens_mask=True,)  
train_dataset = train_dataset.map(encode, batched=True, remove_columns=["Text"])
print(train_dataset[19634])
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "special_tokens_mask"])
print(train_dataset[19634])

In [ ]:
#!g1.1
print([tokenizer.decode(x) for x in train_dataset[19634]['input_ids']])

Задание параметров обучения

In [ ]:
#!g1.1
#pragma async
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True)
training_args = TrainingArguments(
    "/home/jupyter/mnt/s3/model-output",
    num_train_epochs=30,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=16,
    warmup_steps=10,
    weight_decay=0.05
)
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset, 
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None)
)
trainer.train()